# Decision Trees

## Short Answer

1. What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with 1 million instances?

    * depth = log2(1,000,000) = ~20; CART is a binary algorithm.

2. Is a node’s Gini impurity generally lower or greater than its parent’s? Is it generally lower/greater, or always lower/greater?

    * A node's gini impurity is generally lower than its parent's. This is because the cost function for the CART training      algorithm splits children nodes in a way that minimizes the sum of weighted impurity. However, it is possible to have     a higher child impurity if the other child pair is small enough.  

3. If a Decision Tree is overfitting the training set, is it a good idea to try decreasing max_depth?

    * Yes. Reducing the depth of the tree (and leaves/nodes) will help generalize the model by increasing the predicted         sample size averages. 

4. If a Decision Tree is underfitting the training set, is it a good idea to try scaling the input features?

    * No. Decision trees do not repsond to feature scaling. However, you can increase max_depth and reduce min_leaf_samples     to help imporve model fit. 

5. If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?

    * time = (1/1,000,000)*10,000,000 = ~10

6. If your training set contains 100,000 instances, will setting presort=True speed up training?

    * Yes, but this parameter has been depreciated.



## Hands On

7. Train and fine-tune a Decision Tree for the moons dataset. 
    * Generate a moons dataset using make_moons(n_samples=10000, noise=0.4). 
    * Split it into a training set and a test set using train_test_split().
    * Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values        for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes. 
    * Train it on the full training set using these hyperparameters, and measure your model’s performance on the test           set. You should get roughly 85% to 87% accuracy. 

In [2]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4, random_state=42)

print(X.shape)
print(y.shape)

(10000, 2)
(10000,)


In [3]:
from sklearn.model_selection import train_test_split
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(len(X_train))

8000


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
# grid search wraps the tree classifier
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, verbose=1, cv=3)

# this is the trained model
grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 294 candidates, totalling 882 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 882 out of 882 | elapsed:    6.1s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=42,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16

In [5]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=17,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [6]:
from sklearn.metrics import accuracy_score

# make a prediction with the trained model
y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8695

8. Grow a forest.
    * Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use ScikitLearn’s ShuffleSplit class for this.
    * Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.
    * Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This gives you majority-vote predictions over the test set. 
    * Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). 

*Congratulations, you have trained a Random Forest classifier!*

In [16]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

shuffle = ShuffleSplit(n_splits=n_tree_subsets, test_size=len(X_train)-n_instances, random_state=42)

mini_sets = []

for mini_train_index, mini_test_index in shuffle.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [22]:
x = mini_sets[0]
x[0].shape
x[1].shape

(100,)

In [33]:
from sklearn.base import clone
import numpy as np

forest = [clone(grid_search_cv.best_estimator_) for tree in range(n_trees)]

accuracy_scores = []

# zip decision tree model to each tree subset. minisets is a list of nested arrays, note how they are accessed in the for loop.
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.8054499999999999

In [54]:
print(y_pred.shape)
print(X_test.shape)

(2000,)
(2000, 2)


In [37]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

# make prediction with each decision tree model in the forest
for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)



In [44]:
Y_pred.shape

(1000, 2000)

In [38]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

In [56]:
n_votes.shape

(1, 2000)

## Basic Flow
1. import train_test_split, grid_search_cv, decision_tree_classifier, and accuracy score
2. import data and split into test and training groups 
3. choose parameters to search, instantiate grid_search_cv, and wrap decision_tree_classifier
4. fit grid_search_cv, make a prediction, and print scores
